# Imports

In [2]:
import pandas as pd
import numpy as np

# Load Data

In [3]:
data = pd.read_csv('../data/transportation_world.csv', encoding='unicode_escape', sep=';', header=None).drop(0, axis=1)
data.columns = ['Duration', 'Type', 'Year', 'Vacations']
data['Vacations'] = data['Vacations'].apply(lambda x: 0 if x=='..' else int(x))

In [4]:
data

,Duration,Type,Year,Vacations
0,Mindre end fire overnatninger,Bil,2017,1074671
1,Mindre end fire overnatninger,Bil,2018,928717
2,Mindre end fire overnatninger,Bil,2019,779540
3,Mindre end fire overnatninger,Bil,2020,610632
4,Mindre end fire overnatninger,Bil,2021,477843
...,...,...,...,...
107,Mindst fire overnatninger,Andre transportmidler,2019,28908
108,Mindst fire overnatninger,Andre transportmidler,2020,5348
109,Mindst fire overnatninger,Andre transportmidler,2021,3896
110,Mindst fire overnatninger,Andre transportmidler,2022,20914


In [5]:
data_long = data[data['Duration'] == 'Mindst fire overnatninger'].drop('Duration', axis=1)

# Transportation across time and type

In [6]:
data_long = data_long.pivot(index='Year', columns='Type', values='Vacations')
data_long = data_long.reset_index().drop(['Cykel', 'Sejlbåd'], axis=1)
data_long = data_long.rename_axis(None, axis=1)
data_long['Year'] = data_long['Year'].astype(str)
data_long = data_long[['Year','Fly', 'Bil', 'Tog', 'Skib', 'Bus', 'Andre transportmidler']]

In [7]:
data_long = data_long.rename(columns={'Bil': 'Car', 'Tog': 'Train', 'Skib': 'Ship', 'Fly': 'Airplane', 'Andre transportmidler': 'Other forms of transportation'})

In [8]:
transportation_types = list(data_long.columns[1:])
transportation_types

['Airplane', 'Car', 'Train', 'Ship', 'Bus', 'Other forms of transportation']

In [10]:
from bokeh.io import show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.palettes import Category20, GnBu


# Ensure numeric types
#for col in data_long.columns[1:]:  # skip 'Year' if it's a string category
#    data_long[col] = pd.to_numeric(data_long[col], errors='coerce')
#print(data_long['Year'].values)
# Categories to stack

# Create the source
source = ColumnDataSource(data_long)

# Set up the figure
p = figure(x_range=data_long["Year"], height=600, width=900, title="Forms of transport for vacation",
           toolbar_location=None, tools="hover", tooltips="$name: @$name")

# Assign colors (Bokeh palettes are limited, so repeat if needed)
colors = Category20[len(transportation_types)]
colors = GnBu[len(transportation_types)]

# Add stacked bars
p.vbar_stack(transportation_types, x='Year', width=0.9, color=colors, source=source, legend_label=transportation_types)

# Style tweaks
p.y_range.start = 0
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None


p.legend.click_policy="mute" ### assigns the click policy (you can try to use ''hide'
p.legend.location = "center"

########################################################
# This line puts the legend outside of the plot area

p.add_layout(p.legend[0], 'left')


# Show the plot
show(p)


In [103]:
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource, Select, CustomJS, HoverTool, NumeralTickFormatter
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.palettes import PuBu, BuGn, PuBuGn
import pandas as pd


def load_and_prepare(filepath):
    df = pd.read_csv(filepath, encoding='unicode_escape', sep=';', header=None).drop(0, axis=1)
    df.columns = ['Duration', 'Type', 'Year', 'Vacations']
    df['Vacations'] = df['Vacations'].apply(lambda x: 0 if x == '..' else int(x))
    df = df[df['Duration'] == 'Mindst fire overnatninger'].drop('Duration', axis=1)
    df = df.pivot(index='Year', columns='Type', values='Vacations').reset_index()
    df = df.drop(columns=['Cykel', 'Sejlbåd'], errors='ignore')
    df = df.rename(columns={
        'Bil': 'Car', 'Tog': 'Train', 'Skib': 'Ship',
        'Fly': 'Airplane', 'Andre transportmidler': 'Other'
    })
    df['Year'] = df['Year'].astype(str)
    ordered_cols = ['Year', 'Airplane', 'Car', 'Train', 'Ship', 'Bus', 'Other']
    df = df[[col for col in ordered_cols if col in df.columns]]
    return df


# Load datasets
world_df = load_and_prepare('../data/transportation_world.csv')
dk_df = load_and_prepare('../data/transportation_dk.csv')

transportation_types = list(world_df.columns[1:])
#color_indices = [3, 1, 7, 5, 8, 0]
#colors = [PuBu[max(PuBu.keys())][i] for i in color_indices]
colors = [
    PuBu[9][1],
    BuGn[9][4],
    BuGn[9][1],
    PuBu[9][4],
    PuBuGn[9][6],
    PuBuGn[9][1],
]# Sources
source = ColumnDataSource(world_df)
source_world = ColumnDataSource(world_df)
source_dk = ColumnDataSource(dk_df)

# Create figure
p = figure(x_range=world_df["Year"], height=500, sizing_mode="stretch_width",
           title="Forms of transport for vacation",
           toolbar_location=None, tools="hover", tooltips="$name: @$name")

p.vbar_stack(transportation_types, x='Year', width=0.9, color=colors, source=source, legend_label=transportation_types)

hover = HoverTool(tooltips=[
    ("Transport type", "$name"),
    ("Year", "@Year"),
    ("Vacations", "@$name{0,0}")
])
p.tools = [hover]

p.y_range.start = 0
p.y_range.end = 5_500_000
p.yaxis.formatter = NumeralTickFormatter(format="0,0")
p.yaxis.axis_label = "Number of vacations"
p.xgrid.grid_line_color = None
p.xaxis.axis_label = "Year"
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "center"
p.add_layout(p.legend[0], 'left')

# Dropdown select
select = Select(title="Destination", value="World", options=["World", "Denmark"])

# JS callback
select.js_on_change("value", CustomJS(args=dict(source=source,
                                                source_world=source_world,
                                                source_dk=source_dk),
    code="""
    const selected = cb_obj.value;
    const new_data = selected === 'World' ? source_world.data : source_dk.data;
    const updated = {};
    for (const key in new_data) {
        updated[key] = [...new_data[key]];
    }
    source.data = updated;
    source.change.emit();
"""))

# Export to HTML
output_file("../assets_own/transportation_bokeh_plot.html", title="Vacation Transportation")

# Show with responsive layout
show(column(select, p, sizing_mode="stretch_width"))
